# The Solow model with human capital

In this model project we solve the Solow model with human capital. This is done with the usual Cobb-Douglas function but we also further extend the model with the CES production function.

# The model

Consider the following Solow model with human capital:

\begin{align}
Y_t & = K_t^{\alpha} H_t ^\varphi (A_t L_t)^{1-\alpha}, &0<\alpha,\varphi<1, \ \alpha+\varphi < 1 \\
I_t^K & = s_K Y_t, &0<s_H<1 \\
I_t^H & = s_H Y_t, &0<s_K<1 \\
K_{t+1} & = I_t^K + (1-\delta)K_t, &H_0 \\
H_{t+1} & = I_t^H + (1-\delta)K_t, &K_0 \\
L_{t+1} & = (1+n) L_t, &L_0 \\
A_{t+1} & = (1+g) A_t, &A_0
\end{align}
where $s_h + s_k < 1$.

It can be shown that the model can be translated into the following Solow equations:

\begin{align}
\tilde{k}_{t+1} - \tilde{k}_{t} &= \frac{1}{(1+n)(1+g)}(s_K\tilde{k}_t^{\alpha}\tilde{h}_t^{\varphi}-(n+g+\delta + ng)\tilde{k}_t) \\
\tilde{h}_{t+1} - \tilde{h}_{t} &= \frac{1}{(1+n)(1+g)}(s_H\tilde{k}_t^{\alpha}\tilde{h}_t^{\varphi}-(n+g+\delta + ng)\tilde{h}_t),
\end{align}

where $\tilde{k}_t \equiv \frac{K_t}{A_t L_t}$ and $\tilde{h}_t \equiv \frac{H_t}{A_t L_t}$. In steady state it must hold that $\tilde{k}_{t+1}-\tilde k_t = \tilde{h}_{t+1} - \tilde h_t = 0$.
We utilize this to solve for the steady state values of real and human capital per effective worker, $\tilde{k}^*$ and $\tilde{h}^*$.

To solve the model in Python, we first import the relevant packages.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import optimize
import sympy as sm
sm.init_printing(use_unicode=True)
from IPython.display import display, Latex, Markdown
import ipywidgets as widgets
from ipywidgets import interact

## Steady state in the Solow model with human capital

We can now solve the model analytically using Sympy. First we define the symbols which are to be used. Then we define the two equations $\tilde{k}_{t+1}=\tilde k_t = \tilde k^*$ and $\tilde{h}_{t+1} = \tilde h_t = \tilde h^*$ which are the steady state conditions. Finally, we solve for $\tilde k_t^*$ and $\tilde h_t^*$, respectively.

In [3]:
# Defining the symbols used
sk, sh, alpha, g, delta, n, phi, tilkt, tilkt1, tilht, tilht1 = sm.symbols('s_K s_H alpha g delta n \\varphi \\tilde{k}_t \\tilde{k}_{t+1} \\tilde{h}_t \\tilde{h}_{t+1}')

# Defining the equation for physical capital per effective worker to be solved
ss_k = sm.Eq(1/((1+n)*(1+g))*(sk*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilkt))

# Defining the equation for human capital per effective worker to be solved
ss_h = sm.Eq(1/((1+n)*(1+g))*(sh*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilht))

# Solution for the steady steady state value for ss_k and ss_h
ss_sol = sm.solve((ss_k,ss_h),(tilkt,tilht))
# Steady state value for ss_k
ktilstar = ss_sol[0][0]

# Steady state value for ss_h
htilstar = ss_sol[0][1]

# Latex print of ktilstar
display(
        Markdown(
            r'The steady state value of physical capital per effective worker is $\ \tilde k^*= $ {}.'.format(
                sm.latex(ktilstar, mode='inline'))))

# Latex print of htilstar
display(
    Markdown(
        r'The steady state value of human capital per effective worker is $\ \tilde h^*= $ {}.'.format(
            sm.latex(htilstar, mode='inline'))))

The steady state value of physical capital per effective worker is $\ \tilde k^*= $ $s_{H}^{- \frac{\varphi}{\varphi + \alpha - 1}} \left(s_{K}^{\varphi - 1} \left(\delta + g n + g + n\right)\right)^{\frac{1}{\varphi + \alpha - 1}}$.

The steady state value of human capital per effective worker is $\ \tilde h^*= $ $\left(\left(s_{H}^{- \frac{\varphi}{\varphi + \alpha - 1}} \left(s_{K}^{\varphi - 1} \left(\delta + g n + g + n\right)\right)^{\frac{1}{\varphi + \alpha - 1}}\right)^{- \alpha} \left(\delta + g n + g + n\right) / s_{H}\right)^{\frac{1}{\varphi - 1}}$.

### Parameter calibration

In the following section we insert parameters in the steady state solutions, thus giving a numerical solution. By default we will set the paramaters in the derived steady state values as, $\varphi = \alpha = \frac{1}{3}$, $\delta = 0.05$, $g = n = 0.02$, $s_K = 0.1$ and $s_H = 0.15$. These values can be changed by the user below (given that the parameter restrictions are respected), such that one can see how different parameter values change the steady state values.

In [16]:
# Turn symbols into variables with SymPy's lambdify function
_htilstar_sol = sm.lambdify((sh,phi,alpha,sk,delta,g,n),htilstar)
_ktilstar_sol = sm.lambdify((sh,phi,alpha,sk,delta,g,n),ktilstar)

# Define widget for steady state value
def ss_sol_int(sh,phi,alpha,sk,delta,g,n):
    
    # The sum of the elasticities of human and physical capital AND the sum of the savings rates must be strictly less than 1
    if alpha+phi >= 1.0 and sh+sk >= 1.0:
        result = display(Latex(
            r'$\alpha + \varphi$ and $s_H+s_K$ have to be stricly smaller than 1.0')) #print warning
    # The sum of the savings' rates must be strictly less than 1
    elif sh+sk >= 1.0:
        result = display(Latex(
            r'$s_H+s_K$ has to be stricly smaller than 1.0')) #print warning
    # The elasticities of human and physical capital must som to be strictly less than 1
    elif alpha+phi >= 1.0:
        result = display(Latex(
            r'$\alpha + \varphi$ has to be stricly smaller than 1.0')) #print warning 
    # Stability constraint
    elif n+g+delta+n*g <= 0:
        result = display(Latex(
        r'$n+g+\delta+ng$ has to be strictly greater than $0$.')) #print warning
        
    # If all the above conditions are not violated, print result
    else:
        result = display(Markdown(
            r'The steady state value of human capital per effective worker is $\ \tilde k^*= $ {}.'.format(
            sm.latex(_ktilstar_sol(sh,phi,alpha,sk,delta,g,n), mode='inline'))))
        result = display(Markdown(
            r'The steady state value of human capital per effective worker is $\ \tilde h^*= $ {}.'.format(
            sm.latex(_htilstar_sol(sh,phi,alpha,sk,delta,g,n), mode='inline'))))
    return result

# Make the solution interactive with widget controlling the parameters
widgets.interact(ss_sol_int,
                 sh = widgets.FloatSlider(description='$s_H$', min=0.01, max=0.99, value=0.4, step=0.01, continuous_update=False),
                 phi = widgets.FloatSlider(description='$\\varphi$', min=0.01, max=0.99, value=1/3, step=0.01, continuous_update=False),
                 alpha = widgets.FloatSlider(description='$\\alpha$', min=0.01, max=0.99, value=1/3, step=0.01, continuous_update=False),
                 sk = widgets.FloatSlider(description='$s_K$', min=0.01, max=0.99, value=0.4, step=0.01, continuous_update=False),
                 delta = widgets.FloatSlider(description='$\delta$', min=0.0, max=1.0, value=0.05, step=0.01, continuous_update=False),
                 g = widgets.FloatSlider(description='$g$', min=-1.0, max=1.0, value=0.02, step=0.01, continuous_update=False),
                 n = widgets.FloatSlider(description='$n$', min=-1.0, max=1.0, value=0.02, step=0.01, continuous_update=False)
                )

interactive(children=(FloatSlider(value=0.4, continuous_update=False, description='$s_H$', max=0.99, min=0.01,…

<function __main__.ss_sol_int(sh, phi, alpha, sk, delta, g, n)>

## Transition equations and phase diagram

In the following section we will derive the transition equations for the physical capital, $\tilde k_{t+1}$ and human capital, $\tilde h_{t+1}$.

In [5]:
# Define transition equations for physical and human capital
ktplus1 = (1/((1+n)*(1+g))*(sk*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilkt) + tilkt)
htplus1 = (1/((1+n)*(1+g))*(sh*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilht) + tilht)

# Turn into function with inputs using lambdify function from SymPy
ktplus1lamb = sm.lambdify((tilkt, tilht, alpha, n, g, sh, sk, delta, phi),ktplus1)
htplus1lamb = sm.lambdify((tilkt, tilht, alpha, n, g, sh, sk, delta, phi),htplus1)

# Turn lambdify function into Python function with standard parameterization
def kt_plus1_func(tilkt,tilht,alpha=1/3,n=0.02,g=0.02,sh=0.4,sk=0.4,delta=0.05,phi=1/3):
    return ktplus1lamb(tilkt, tilht, alpha, n, g, sh, sk, delta, phi)
def ht_plus1_func(tilkt,tilht,alpha=1/3,n=0.02,g=0.02,sh=0.4,sk=0.4,delta=0.05,phi=1/3):
    return htplus1lamb(tilkt, tilht, alpha, n, g, sh, sk, delta, phi)

To create the paths where either $\Delta h_t = h_{t+1}-h_t = 0$ or $\Delta k_t = k_{t+1}-k_t = 0$, we isolate $h_t$ in both capital accumulation equations. These are the phase equations, i.e. when the Solow equations are set equal to 0.

In [6]:
# Define Solow equations for physical and human capital
solow_kt = (1/((1+n)*(1+g))*(sk*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilkt))
solow_ht = (1/((1+n)*(1+g))*(sh*tilkt**alpha*tilht**phi-(n+g+delta+n*g)*tilht))

# Solve for physical and human capital when Solow equations are equal to 0
Delta_kt0 = sm.solve(solow_kt,tilht)
Delta_ht0 = sm.solve(solow_ht,tilht)

# Turn into function with inputs using lambdify function from SymPy
Delta_kt0 = sm.lambdify((tilkt, alpha, n, g, sh, sk, delta, phi),Delta_kt0[0])
Delta_ht0 = sm.lambdify((tilkt, alpha, n, g, sh, sk, delta, phi),Delta_ht0[0])

# Turn lambdify function into Python function with standard parameterization
def Delta_kt0_func(tilkt,alpha=1/3,n=0.02,g=0.02,sh=0.4,sk=0.4,delta=0.05,phi=1/3):
    return Delta_kt0(tilkt, alpha, n, g, sh, sk, delta, phi)
def Delta_ht0_func(tilkt,alpha=1/3,n=0.02,g=0.02,sh=0.4,sk=0.4,delta=0.05,phi=1/3):
    return Delta_ht0(tilkt, alpha, n, g, sh, sk, delta, phi)

We create the paths by iterating through a linearly spaced vector (```np.linspace```) from $0.0001$ to $1000$, creating $100000$ evenly spaced numbers.
These values are our given $k_t$, and thus we can find both paths.

In [7]:
# Create plot of convergence to steady state
def create_cd_plot(k0cd_list,h0cd_list,alpha, n, g, sh, sk, delta, phi):
    
    ### Inputs: Lists of k_t and h_t's paths, and parameter values

    # Plot both 
    fig,ax = plt.subplots() # Create plot
    plt.plot(k0cd_list,h0cd_list) #plot lists with paths
    plt.plot(k0cd_list[-1],h0cd_list[-1], 'bo') #plot point with steady state
    #set limits, if the initial values are above steady state, choose these as those who guide the limits
    #if the initial values are below steady state, choose steady state values to guide the limits (last element in list)
    ax.set_xlim([0,max(k0cd_list[-1]*1.1,h0cd_list[-1]*1.1,k0cd_list[0]*1.1,h0cd_list[0]*1.1)]) #set xlim depending on the largest value
    ax.set_ylim([0,max(k0cd_list[-1]*1.1,h0cd_list[-1]*1.1,k0cd_list[0]*1.1,h0cd_list[0]*1.1)]) #set ylim depending on the largest value
    ax.set_title('Convergence in the Solow model with CD production function', pad=15) #Set title
    ax.set_xlabel('$k_t$') #set xlabel
    ax.set_ylabel('$h_t$') #set ylabel
    
    # Define the linearly spaced vector from 0.0001 to 10000 and make 100000 iterations
    x = np.linspace(0.0001,10000,num=100000) 
    Delta_kt0_list = []
    Delta_ht0_list = []
    for i in range(0,x.shape[0]):
        Delta_kt0_list.append(Delta_kt0_func(x[i],alpha, n, g, sh, sk, delta, phi)) #list where Delta k_t = 0
        Delta_ht0_list.append(Delta_ht0_func(x[i],alpha, n, g, sh, sk, delta, phi)) ##list where Delta k_t = 0
    
    plt.plot(x,Delta_kt0_list) #plot loci
    plt.plot(x,Delta_ht0_list) #plot loci
    return plt.show() #return plot

To make the plot interactive we now define a function, ```interact_CD```, where the user can define the paramters which are to be used. The plot then shows the convergence path.

In [8]:
# Define interactive part
def interact_CD(k0=10, h0=10,sk=0.2,sh=0.1,alpha=1/3,phi=1/3,n=0.02,g=0.02,delta=0.05):
    
    # Make same conditions as with steady state values
    if sk + sh >= 1 and alpha + phi >=  1:
        result = display(Latex(r'$\alpha + \varphi$ and $s_H + s_K$ has to be strictly less than $1$.')) #print warning
    elif alpha + phi >= 1:
        result = display(Latex(r'$\alpha + \varphi$ has to be stricly less than $1$.')) #print warning
    elif sk + sh >= 1:
        result = display(Latex(r'$s_H+s_K$ has to be stricly less than $1$.')) #print warning
    elif (n + g + delta+n*g) <= 0:
        result = display(Latex(r'$n+g+\delta$ has to be greater than $0$.')) #print warning
     
    # Else create plot
    else:    
        
        # Initialize lists that will hold all the values of k_t and h_t, starting with k_0 and h_0
        k0cd_list = [k0] 
        h0cd_list = [h0]
        # Append k_1 and h_1 to their respective lists. They are calculated using the functions defined earlier, here hardreferencing to the initial values
        k0cd_list.append(kt_plus1_func(k0cd_list[0],h0cd_list[0],alpha, n, g, sh, sk, delta, phi))
        h0cd_list.append(ht_plus1_func(k0cd_list[0],h0cd_list[0],alpha, n, g, sh, sk, delta, phi))
        i = 1 #initialize i for while loop
        while i<10000: #while loop that calculates the next values...
            if ((np.abs(h0cd_list[-1]-h0cd_list[-2])<0.00001) & (np.abs(k0cd_list[-1]-k0cd_list[-2])<0.00001)): #... unless the error margin has been reach
                display(Latex('Convergence archieved!')) #Inform of convergence
                result = create_cd_plot(k0cd_list,h0cd_list,alpha, n, g, sh, sk, delta, phi) # As convergence has been reached, create plot
                break
            elif i==9999: #... or the max iterations have been reached
                result = display(Latex('No convergence archieved')) #Inform of no convergence :(
                break 
            else: #If error margin not satisfied and max iter not reached, append new values calculated using function and the last value of the lists, repeat while loop
                k0cd_list.append(kt_plus1_func(k0cd_list[i],h0cd_list[i],alpha, n, g, sh, sk, delta, phi)) 
                h0cd_list.append(ht_plus1_func(k0cd_list[i],h0cd_list[i],alpha, n, g, sh, sk, delta, phi))
                i += 1 #increase such that new values are used
    return result

# Let plot and sliders interact
widgets.interact(interact_CD,   
            k0    = widgets.FloatSlider(description='$k_0$', min=1, max=10,step=0.25,value=4,continuous_update=False),
            h0    = widgets.FloatSlider(description='$h_0$', min=1, max=15,step=0.25,value=6,continuous_update=False),
            sk    = widgets.FloatSlider(description='$s_K$', min=0.01, max=0.99,step=0.01,value=0.1,continuous_update=False),
            sh    = widgets.FloatSlider(description='$s_H$', min=0.01, max=0.99,step=0.01,value=0.15,continuous_update=False),
            alpha = widgets.FloatSlider(description='$\\alpha$', min=0, max=0.99,step=0.01,value=1/3,continuous_update=False),
            phi   = widgets.FloatSlider(description='$\\varphi$', min=0, max=0.99,step=0.01,value=1/3,continuous_update=False),
            n     = widgets.FloatSlider(description='$n$', min=-1, max=1,step=0.01,value=0.02,continuous_update=False),
            g     = widgets.FloatSlider(description='$g$', min=-1, max=1,step=0.01,value=0.02,continuous_update=False),
            delta = widgets.FloatSlider(description='$\delta$', min=0, max=1,step=0.01,value=0.05,continuous_update=False)
                )

interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='$k_0$', max=10.0, min=1.0, …

<function __main__.interact_CD(k0=10, h0=10, sk=0.2, sh=0.1, alpha=0.3333333333333333, phi=0.3333333333333333, n=0.02, g=0.02, delta=0.05)>

# The Solow model with human capital and a CES production function

In the following section we will extend the Solow model by changing the Cobb-Douglas production function with a CES (constant elasticity of substitution) function. We then have the following model setup.

\begin{align}
Y_t & = \left[\alpha K_t^{\frac{\sigma-1}{\sigma}}+ \varphi H_t^{\frac{\sigma-1}{\sigma}} + (1-\alpha-\varphi)(A_tL_t)^{\frac{\sigma-1}{\sigma}}\right]^{\frac{\sigma}{\sigma-1}}, \ &\alpha+\varphi < 1, \ \sigma \geq 0 \\
I_t^K & = s_K Y_t, &0<s_H<1 \\
I_t^H & = s_H Y_t, &0<s_K<1 \\
K_{t+1} & = I_t^K + (1-\delta)K_t, &H_0 \\
H_{t+1} & = I_t^H + (1-\delta)K_t, &K_0 \\
L_{t+1} & = (1+n) L_t, &L_0 \\
A_{t+1} & = (1+g) A_t, &A_0
\end{align}
where $s_h + s_k < 1$.


This is done to consider the more general case for the model as the Cobb-Douglas function is merely a special case of the CES function (that is, for $\sigma \to 1$ the CES production function corresponds to a Cobb-Douglas production function).

**NB:** SymPy cannot solve the model analytically with the CES production function. Hence, the model will only be solved numerically in this section.

Firstly, we define the two transition equations used:

\begin{align}
\tilde k_{t+1} &= \frac{1}{(1+n)(1+g)} \left(\left(s_K \left(\alpha \tilde k_t^{\frac{\sigma-1}{\sigma}} + \varphi \tilde h_t^{\frac{\sigma-1}{\sigma}} + (1-\alpha-\varphi) \right)\right)^{\frac{\sigma}{\sigma-1}}-(n+g+\delta+ng)\tilde k_t \right) + \tilde k_t \\
\tilde h_{t+1} &= \frac{1}{(1+n)(1+g)} \left(\left(s_H \left(\alpha \tilde k_t^{\frac{\sigma-1}{\sigma}} + \varphi \tilde h_t^{\frac{\sigma-1}{\sigma}} + (1-\alpha-\varphi) \right)\right)^{\frac{\sigma}{\sigma-1}}-(n+g+\delta+ng)\tilde h_t \right) + \tilde h_t
\end{align}

In [9]:
# Define transition equation for physical capital per effective worker
def kt1(tilkt,tilht,sk=0.4,alpha=1/3,phi=1/3,sigma=1.5,n=0.02,g=0.02,delta=0.05):
    return (1/((1+n)*(1+g)))*(sk*(alpha*tilkt**((sigma-1)/sigma)+phi*tilht**((sigma-1)/sigma)+(1-alpha-phi))**(sigma/(sigma-1))-(n+g+delta+n*g)*tilkt+tilkt)

# Define transition equation for human capital per effective worker
def ht1(tilkt,tilht,sh=0.2,alpha=1/3,phi=1/3,sigma=1.5,n=0.02,g=0.02,delta=0.05):
    return (1/((1+n)*(1+g)))*(sh*(alpha*tilkt**((sigma-1)/sigma)+phi*tilht**((sigma-1)/sigma)+(1-alpha-phi))**(sigma/(sigma-1))-(n+g+delta+n*g)*tilht)+tilht

In [ ]:
  # Initialize lists that will hold all the values of k_t and h_t, starting with k_0 and h_0
        k0cd_list = [k0] 
        h0cd_list = [h0]
        # Append k_1 and h_1 to their respective lists. They are calculated using the functions defined earlier, here hardreferencing to the initial values
        k0cd_list.append(kt_plus1_func(k0cd_list[0],h0cd_list[0],alpha, n, g, sh, sk, delta, phi))
        h0cd_list.append(ht_plus1_func(k0cd_list[0],h0cd_list[0],alpha, n, g, sh, sk, delta, phi))
        i = 1 #initialize i for while loop
        while i<10000: #while loops that calculates the next values...
            if ((np.abs(h0cd_list[-1]-h0cd_list[-2])<0.00001) & (np.abs(k0cd_list[-1]-k0cd_list[-2])<0.00001)): #... unless the error margin has been reach
                display(Latex('Convergence archieved!')) #Inform of convergence
                result = create_cd_plot(k0cd_list,h0cd_list,alpha, n, g, sh, sk, delta, phi) # As convergence has been reached, create plot
                break
            elif i==9999: #if reached
                result = display(Latex('No convergence archieved')) #Inform of no convergence :(
                break 
            else: #If error margin not satisfied, append new values calculated using function and the last value of the lists, repeat while loop
                k0cd_list.append(kt_plus1_func(k0cd_list[i],h0cd_list[i],alpha, n, g, sh, sk, delta, phi)) 
                h0cd_list.append(ht_plus1_func(k0cd_list[i],h0cd_list[i],alpha, n, g, sh, sk, delta, phi))
                i += 1 #increase such that new values are used

In [18]:
# Define interactive part
def interact_CES(k0=10, h0=10,sk=0.2,sh=0.1,alpha=1/3,phi=1/3,sigma=1.5,n=0.02,g=0.02,delta=0.05):
    ### Check if conditions are violated ###
    # sum of savings rates and sum of income shares must not exceed 1
    if sk + sh >= 1 and alpha + phi >=  1:
        result = display(Latex(r'$\alpha + \varphi$ and $s_H + s_K$ has to be strictly less than $1$.')) #print warning
    # sum of income shares must not exceed 1
    elif alpha + phi >= 1:
        result = display(Latex(r'$\alpha + \varphi$ has to be stricly less than $1$.')) #print warning
    # sum of savings rates must not exceed 1
    elif sk + sh >= 1:
        result = display(Latex(r'$s_H+s_K$ have to be stricly less than $1$.')) #print warning
    elif (n + g + delta) == 0: 
        result = display(Latex(r'$n+g+\delta$ has to be greater than $0$. They have been reset to $n=g=\delta=0.05$.')) #print warning
    else:    
        # Initialize lists that will hold all the values of k_t and h_t, starting with k_0 and h_0
        k0ces_list = [k0]
        h0ces_list = [h0]
        # Append k_1 and h_1 to their respective lists. They are calculated using the functions defined earlier, here hardreferencing to the initial values
        k0ces_list.append(kt1(k0ces_list[0],h0ces_list[0],sk,alpha,phi,sigma,n,g,delta))
        h0ces_list.append(ht1(k0ces_list[0],h0ces_list[0],sh,alpha,phi,sigma,n,g,delta))
        i = 1 #initialize i for while loop
        while i<10000: #while loop that calculates the next values...
            if ((np.abs(h0ces_list[-1]-h0ces_list[-2])<0.00001) & (np.abs(k0ces_list[-1]-k0ces_list[-2])<0.00001)): #... unless the error margin has been reached
                display(Latex('Convergence archieved!')) #Inform of convergence
                result = create_ces_plot(k0ces_list,h0ces_list) # As convergence has been reached, create plot
                break
            elif i==9999: #... unless the max iter has been reached
                result = display(Latex('No convergence archieved')) #Inform of no converge :(
                break
            else: #If error margin not satisfied and max iter not reached, append new values calculated using function and the last value of the lists, repeat while loop
                k0ces_list.append(kt1(k0ces_list[i],h0ces_list[i],sk,alpha,phi,sigma,n,g,delta))
                h0ces_list.append(ht1(k0ces_list[i],h0ces_list[i],sh,alpha,phi,sigma,n,g,delta))
                i += 1 #increase i for while loop
    return result

In [19]:
# Create plot of convergence to steady state
def create_ces_plot(k0ces_list,h0ces_list):
    ###Inputs: List of paths of k_t and h_t
    fig,ax = plt.subplots() #create plot
    plt.plot(k0ces_list,h0ces_list) #plot the lists of k_t and h_t
    plt.plot(k0ces_list[-1],h0ces_list[-1], 'bo') #plot a point with the steady state values
    #set limits, if the initial values are above steady state, choose these as those who guide the limits
    #if the initial values are below steady state, choose steady state values to guide the limits (last element in list)
    ax.set_xlim([0,max(k0ces_list[-1]*1.1,h0ces_list[-1]*1.1,k0ces_list[0]*1.1,h0ces_list[0]*1.1)])
    ax.set_ylim([0,max(k0ces_list[-1]*1.1,h0ces_list[-1]*1.1,k0ces_list[0]*1.1,h0ces_list[0]*1.1)])
    ax.set_title('Convergence in the Solow model with CES production function', pad=15) #Set title
    ax.set_xlabel('$k_t$') #Set xlabel
    ax.set_ylabel('$h_t$') #Set ylabel
    return plt.show()


#Let plot and sliders interact
widgets.interact(interact_CES,   
            k0    = widgets.FloatSlider(description='$k_0$', min=1, max=10,step=0.25,value=4,continuous_update=False),
            h0    = widgets.FloatSlider(description='$h_0$', min=1, max=15,step=0.25,value=6,continuous_update=False),
            sk    = widgets.FloatSlider(description='$s_K$', min=0.01, max=0.99,step=0.01,value=0.1,continuous_update=False),
            sh    = widgets.FloatSlider(description='$s_H$', min=0.01, max=0.99,step=0.01,value=0.15,continuous_update=False),
            alpha = widgets.FloatSlider(description='$\\alpha$', min=0.01, max=0.99,step=0.01,value=1/3,continuous_update=False),
            phi   = widgets.FloatSlider(description='$\\varphi$', min=0.01, max=0.99,step=0.01,value=1/3,continuous_update=False),
            sigma = widgets.FloatSlider(description='$\sigma$', min=0, max=10,step=0.05,value=1.5,continuous_update=False),
            n     = widgets.FloatSlider(description='$n$', min=-1, max=1,step=0.01,value=0.01,continuous_update=False),
            g     = widgets.FloatSlider(description='$g$', min=-1, max=1,step=0.01,value=0.01,continuous_update=False),
            delta = widgets.FloatSlider(description='$\delta$', min=0, max=1,step=0.01,value=0.05,continuous_update=False)
                )

interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='$k_0$', max=10.0, min=1.0, …

<function __main__.interact_CES(k0=10, h0=10, sk=0.2, sh=0.1, alpha=0.3333333333333333, phi=0.3333333333333333, sigma=1.5, n=0.02, g=0.02, delta=0.05)>